# KVLCC2

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
import matplotlib
#plt.style.use('presentation')

from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig


from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.models.regression import MotionRegression, Regression

from src.parameters import df_parameters
from src.substitute_dynamic_symbols import run
from src.models.diff_eq_to_matrix import DiffEqToMatrix
p = df_parameters["symbol"]
import statsmodels.api as sm

from src.models.force_from_motion import predict_force

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)
from src.prime_system import PrimeSystem
from wPCC_pipeline.pipelines.kvlcc2.nodes import calculate_thrust

| Column | Unit | Meaning | Reference |
| ------ | ---- | ------- | ----------|
| 1 | s | time | manoeuvre starts at zero i.e. the rudder starts to move at t=0. Time step is about 0.135(full scale) |
| 2 | m | x-position | position of ship origo  in Earth-fixed coordinate system(North). |
| 3 | m | y-position | position of ship origo in Earth-fixed coordinate system(East) |
| 4 | deg | heel angle | starboard side in the water is a positive |
| 5 | deg | heading angle | bow to starboard is positive |
| 6 | m/s | long. velocity | speed through the water of ship origo |
| 7 | m/s | transv. velocity | speed through the water of ship origo |
| 8 | deg/s | roll velocity | ship origo, starboard downwards is positive |
| 9 | deg/s | yaw velocity | ship origo, bow towards starboard is positive |
| 10 | deg | rudder angle | trailing edge to portside is positive |
| 11 | RPM | prop. revolutions | positive clockwise seen from aft. |


In [ ]:
columns = [
'time',
'x0',
'y0',
'phi',
'psi',
'u',
'v',
'p',
'r',
'delta',
'rev',
]

df = pd.read_csv("../data/01_raw/kvlcc2/MARIN_FREE_KVLCC2_zz_-20_m.dat", names=columns, header=1, sep='\t')
df.dropna(how='all', inplace=True)

dt = 0.135
df['time'] = np.arange(0,len(df)*dt,dt)


df.set_index('time', inplace=True)
df.sort_index(inplace=True)
angles = ['phi','psi','r','delta']
df[angles] = np.deg2rad(df[angles])

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,15)
ax.axis('equal')
df.plot(x='y0', y='x0', ax=ax)


In [ ]:
df.iloc[-100:].plot(y='x0')

In [ ]:
df.iloc[-100:]

In [ ]:
%reload_kedro
ship_data = catalog.load("kvlcc2.ship_data")

In [ ]:
np.sqrt(ship_data['I_z']/ship_data['m'])/ship_data['L']

In [ ]:
ship_data_wpcc = catalog.load("wpcc.ship_data")
np.sqrt(ship_data_wpcc['I_z']/ship_data_wpcc['m'])/ship_data_wpcc['L']

In [ ]:
ps = PrimeSystem(**ship_data)

In [ ]:
shipdata_prime = ps.prime(ship_data)

In [ ]:
ship_data['m']/(1/2*1000*(ship_data['L']**2*ship_data['T']))

In [ ]:
0.235/(1/2*(2.909**3))

In [ ]:
I_z = ps._unprime(0.011,unit = "inertia_moment")
np.sqrt(I_z/ship_data['m'])/ship_data['L']

In [ ]:
k_z = 0.25
ship_data['m']*(ship_data['L']*k_z)**2

In [ ]:
ship_data['m']*(ship_data['L']*0.5)**2

In [ ]:
prime_1_to_2 = ship_data['T']/ship_data['L']
Xudot_prime2 = 0.022
Yvdot_prime2 = 0.223
Nrdot_prime2 = 0.011

Xudot_prime = Xudot_prime2*prime_1_to_2
Yvdot_prime = Yvdot_prime2*prime_1_to_2
Nrdot_prime = Nrdot_prime2*prime_1_to_2


In [ ]:
Xudot_prime

In [ ]:
Yvdot_prime

In [ ]:
Nrdot_prime

In [ ]:
(mx_prime-shipdata_prime['m'])/shipdata_prime['m']

In [ ]:
X_rudder = 5.2/ship_data['scale_factor'] - ship_data['L']/2
X_rudder

In [ ]:
%reload_kedro

#id = "MARIN_FREE_KVLCC2_tc_35_m"
id = "MARIN_FREE_KVLCC2_zz_20_m"
dataframes = {}
dataframes[id] = catalog.load(f"kvlcc2.{id}.raw_data")

id = "MARIN_FREE_KVLCC2_zz_-20_m"
#df_ = catalog.load(f"kvlcc2.{id}.data_ek_smooth")
df_ = catalog.load(f"kvlcc2.{id}.raw_data")
df_['y0']*=-1
df_['psi']*=-1
df_['delta']*=-1
df_['r']*=-1
df_['v']*=-1
#df_['r1d']*=-1
#df_['v1d']*=-1

dataframes[id] = df_

track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B']);
plot(dataframes);

In [ ]:
%reload_kedro

#id = "MARIN_FREE_KVLCC2_tc_35_m"
id = "MARIN_FREE_KVLCC2_zz_20_m"

df_raw = catalog.load(f"kvlcc2.{id}.raw_data")
data = catalog.load(f"kvlcc2.{id}.data")
df_smooth = catalog.load(f"kvlcc2.{id}.data_ek_smooth")
dataframes = {'raw':data,
              'ek' : df_smooth,
             }

In [ ]:
track_plots(dataframes=dataframes, lpp=ship_data['L'], beam=ship_data['B']);

In [ ]:
plot(dataframes=dataframes);

In [ ]:
%reload_kedro
ship_data = catalog.load("kvlcc2.ship_data")
open_water_characteristics = catalog.load(f"kvlcc2.open_water_characteristics")
open_water_characteristics['Kq'] = open_water_characteristics["10Kq"]/10

In [ ]:
open_water_characteristics.plot(y=['Kt','10Kq','Eta'])

In [ ]:
open_water_characteristics.head()

In [ ]:
coefficients = np.polyfit(x=open_water_characteristics.index, y=open_water_characteristics['Kt'], deg=2)

In [ ]:
df_ = pd.DataFrame()
J = open_water_characteristics.index
n = 1.0
D = ship_data['D']
w0 = ship_data['w0']
rho = 1000
tdf = ship_data['tdf']
df_['J'] = J
df_['u'] = J*n*D/(1-w0)
df_['v'] = 0
df_['r'] = 0
df_['rev'] = n
df_['V'] = np.sqrt(df_['u']**2 + df_['v']**2)

df_ = calculate_thrust(df=df_, ship_data=ship_data, open_water_characteristics=open_water_characteristics)
df_['Kt'] = df_['thrust'] / ((1 - tdf) * rho * n ** 2 * D ** 4)

In [ ]:
fig,ax=plt.subplots()
open_water_characteristics.plot(y='Kt', ax=ax, label='MARIN')
df_.plot(x='J', y='Kt', ax=ax, label='NMRI', style='--')

#ax.plot(J, np.polyval(coefficients, x=J),'--', 'fit')